# Scientific Python: Using SciPy for Optimization
---
---

### Understanding the `scipy` module

In [1]:
import scipy

In [2]:
help(scipy)

Help on package scipy:

NAME
    scipy

DESCRIPTION
    SciPy: A scientific computing package for Python
    
    Documentation is available in the docstrings and
    online at https://docs.scipy.org.
    
    Contents
    --------
    SciPy imports all the functions from the NumPy namespace, and in
    addition provides:
    
    Subpackages
    -----------
    Using any of these subpackages requires an explicit import. For example,
    ``import scipy.cluster``.
    
    ::
    
     cluster                      --- Vector Quantization / Kmeans
     fft                          --- Discrete Fourier transforms
     fftpack                      --- Legacy discrete Fourier transforms
     integrate                    --- Integration routines
     interpolate                  --- Interpolation Tools
     io                           --- Data input and output
     linalg                       --- Linear algebra routines
     linalg.blas                  --- Wrappers to BLAS library
     li

`👉"Using any of these subpackages requires an explicit import.`

### Using the cluster module in SciPy
---

Clustering is a popular technique to categorize data by associating it into groups. The SciPy library includes an implementation of the k-means clustering algorithm as well as several hierarchical clustering algorithms. In this example, you’ll be using the k-means algorithm in `scipy.cluster.vq,` where `vq `stands for vector quantization.

In the dataset, each message has one of two labels:

`ham` for legitimate messages

`spam` for spam messages

The full text message is associated with each label. When you scan through the data, you might notice that spam messages tend to have a lot of numeric digits in them. They often include a phone number or prize winnings. Let’s predict whether or not a message is spam based on the number of digits in the message. To do this, you’ll cluster the data into three groups based on the number of digits that appear in the message:

`Not spam:` Messages with the smallest number of digits are predicted not to be spam.

`Unknown:` Messages with an intermediate number of digits are unknown and need to be processed by more advanced algorithms.

`Spam:` Messages with the highest number of digits are predicted to be spam.

Let’s get started with clustering the text messages. First, you should import the libraries you’ll use in this example:

In [4]:
# import packages
from pathlib import Path
import numpy as np
from scipy.cluster.vq import whiten, kmeans, vq

In this example, there are 5,574 observations, or individual messages, in the dataset. In addition, you’ll see that there are two features:

The number of digits in a text message

The number of times that number of digits appears in the whole dataset

In [5]:
# Read the file as text where class is separated by tab
data = Path('/Users/patrick/Desktop/Lighthouse_labs/Lighthouse-data-notes/Unit_5/Day_3/smsspamcollection/SMSSpamCollection').read_text()
data = data.strip().split('\n')

We can start analyzing the data

In [6]:
# 1. We need to count the number of digits in each message. But because our function expects a numpy array. we are going to count using np.empty menthod
digit_counts = np.empty((len(data),2), dtype=int)

In [7]:
digit_counts

array([[140630927941480,      4506025408],
       [          61485,              -1],
       [      168627541, 294662192521857],
       ...,
       [              0,               0],
       [              0,               0],
       [              0,               0]])

In this code, you’re creating an empty NumPy array, digit_counts, which has two columns and 5,574 rows. The number of rows is equal to the number of messages in the dataset. You’ll be using digit_counts to associate the number of digits in the message with whether or not the message was spam.

You should create the array before entering the loop, so you don’t have to allocate new memory as your array expands. This improves the efficiency of your code. Next, you should process the data to record the number of digits and the status of the message:

In [8]:
# 2. Now we create the loop to count the digits and the status of the message:
for i, line in enumerate(data):
    case, message = line.split("\t")
    num_digits = sum(c.isdigit() for c in message)
    digit_counts[i, 0] = 0 if case == "ham" else 1
    digit_counts[i, 1] = num_digits

Line 8: Loop over data. You use enumerate() to put the value from the list in line and create an index i for this list. To learn more about enumerate(), check out Python enumerate(): Simplify Looping With Counters.

Line 9: Split the line on the tab character to create case and message. case is a string that says whether the message is ham or spam, while message is a string with the text of the message.

Line 10: Calculate the number of digits in the message by using the sum() of a comprehension. In the comprehension, you check each character in the message using isdigit(), which returns True if the element is a numeral and False otherwise. sum() then treats each True result as a 1 and each False as a 0. So, the result of sum() on this comprehension is the number of characters for which isdigit() returned True.

Line 11: Assign values into digit_counts. You assign the first column of the i row to be 0 if the message was legitimate (ham) or 1 if the message was spam.

Line 12: Assign values into digit_counts. You assign the second column of the i row to be the number of digits in the message.



Now you have a NumPy array that contains the number of digits in each message. However, you want to apply the `clustering algorithm` to an array that has the number of messages with a certain number of digits. In other words, you need to create an array where the first column has the number of digits in a message, and the second column is the number of messages that have that number of digits. Check out the code below:

In [9]:
unique_counts = np.unique(digit_counts[:, 1], return_counts=True)


`np.unique()` takes an array as the first argument and returns another array with the unique elements from the argument. It also takes several optional arguments. Here, you use return_counts=True to instruct np.unique() to also return an array with the number of times each unique element is present in the input array. These two outputs are returned as a tuple that you store in unique_counts.



In [10]:
# Next, you need to transform unique_counts into a shape that’s suitable for clustering:
unique_counts = np.transpose(np.vstack(unique_counts))

In [12]:
unique_counts.shape

(41, 2)

In [13]:
unique_counts

array([[   0, 4110],
       [   1,  486],
       [   2,  160],
       [   3,   78],
       [   4,   42],
       [   5,   39],
       [   6,   16],
       [   7,   14],
       [   8,   28],
       [   9,   17],
       [  10,   16],
       [  11,   34],
       [  12,   30],
       [  13,   31],
       [  14,   37],
       [  15,   29],
       [  16,   35],
       [  17,   33],
       [  18,   41],
       [  19,   47],
       [  20,   18],
       [  21,   31],
       [  22,   28],
       [  23,   36],
       [  24,   34],
       [  25,   16],
       [  26,   16],
       [  27,   13],
       [  28,   19],
       [  29,    9],
       [  30,    2],
       [  31,    6],
       [  32,    3],
       [  33,    4],
       [  34,    3],
       [  35,    4],
       [  36,    1],
       [  37,    1],
       [  40,    4],
       [  41,    2],
       [  47,    1]])

You combine the two 1xN outputs from np.unique() into one 2xN array using np.vstack(), and then transpose them into an Nx2 array. This format is what you’ll use in the clustering functions. Each row in unique_counts now has two elements:

- The number of digits in a message
- The number of messages that had that number of digits

n the dataset, there are 4110 messages that have no digits, 486 that have 1 digit, and so on. Now, you should apply the k-means clustering algorithm to this array:

In [14]:
# 3. Now we can use the kmeans function to cluster the data:
whitened_counts = whiten(unique_counts)
codebook, _ = kmeans(whitened_counts, 3)

You use `whiten()` to normalize each feature to have unit variance, which improves the results from `kmeans().` Then, `kmeans()` takes the whitened data and the number of clusters to create as arguments. In this example, `you want to create 3 clusters,` for `definitely ham`, `definitely spam,` and `unknown.`

kmeans() returns two values:

- An array with three rows and two columns representing the centroids of each group: `The kmeans()` algorithm calculates the optimal location of the centroid of each cluster by minimizing the distance from the observations to each centroid. This array is assigned to codebook.

- The `mean Euclidian distance from the observations to the centroids:` You won’t need that value for the rest of this example, so you can assign it to _.

In [15]:
# 4. Finally, we can use the vq function to assign each observation to a cluster:
codes, _ = vq(whitened_counts, codebook)

`vq()` assigns codes from the codebook to each observation. It returns two values:

- The first value is an array of the same length as unique_counts, where the value of each element is an integer representing which cluster that observation is assigned to. Since you used three clusters in this example, each observation is assigned to cluster 0, 1, or 2.

- The second value is an array of the Euclidian distance between each observation and its centroid.

Now that you have the data clustered, you should use it to make predictions about the SMS messages. You can inspect the counts to determine at how many digits the clustering algorithm drew the line between definitely ham and unknown, and between unknown and definitely spam.

In [16]:
# The clustering algorithm randomly assigns the code 0, 1, or 2 to each cluster, so you need to identify which is which. You can use this code to find the code associated with each cluster:
ham_code = codes[0]
spam_code = codes[-1]
unknown_code = list(set(range(3)) ^ set([ham_code, spam_code]))[0]

In this code, the first line finds the code associated with ham messages. According to our hypothesis above, the ham messages have the fewest digits, and the digit array was sorted from fewest to most digits. Thus, the ham message cluster starts at the beginning of codes.

Similarly, the spam messages have the most digits and form the last cluster in codes. Therefore, the code for spam messages will be equal to the last element of codes. Finally, you need to find the code for unknown messages. Since there are only 3 options for the code and you have already identified two of them, you can use the `symmetric_difference` operator on a `Python set` to determine the last code value. Then, you can print the cluster associated with each message type:

In [17]:
# print the cluster associated with each message type:
print(f"Ham messages are associated with code {ham_code}")
print(f"Spam messages are associated with code {spam_code}")
print(f"Unclassified messages are associated with code {unknown_code}")

Ham messages are associated with code 1
Spam messages are associated with code 0
Unclassified messages are associated with code 2


In [19]:
# print cluster centers and shape
print("definitely ham:", unique_counts[codes == ham_code][-1])
print("definitely spam:", unique_counts[codes == spam_code][-1])
print("unknown:", unique_counts[codes == unknown_code][-1])

definitely ham: [   0 4110]
definitely spam: [47  1]
unknown: [20 18]


In this output, you see that the definitely ham messages are the messages with zero digits in the message, the unknown messages are everything between 1 and 20 digits, and definitely spam messages are everything from 21 to 47 digits, which is the maximum number of digits in your dataset.

Now, you should check how accurate your predictions are on this dataset. First, create some masks for `digit_counts` so you can easily grab the ham or spam status of the messages:

In [28]:
# create some masks for `digit_counts` so you can easily grab the ham or spam status of the messages:
digits = digit_counts[:, 1]
predicted_hams = digits == 0
predicted_spams = digits > 20
predicted_unknowns = np.logical_and(digits > 0, digits <= 20)


In this code, you’re creating the predicted_hams mask, where there are no digits in a message. Then, you create the predicted_spams mask for all messages with more than 20 digits. Finally, the messages in the middle are predicted_unknowns.



Next, apply these masks to the actual digit counts to retrieve the predictions:

In [29]:
spam_cluster = digit_counts[predicted_spams]
ham_cluster = digit_counts[predicted_hams]
unk_cluster = digit_counts[predicted_unknowns]

Here, you’re applying the masks you created in the last code block to the digit_counts array. This creates three new arrays with only the messages that have been clustered into each group. Finally, you can see how many of each message type have fallen into each cluster:

In [30]:
print("hams:", np.unique(ham_cluster[:, 0], return_counts=True))
print("spams:", np.unique(spam_cluster[:, 0], return_counts=True))
print("unknowns:", np.unique(unk_cluster[:, 0], return_counts=True))

hams: (array([0, 1]), array([4071,   39]))
spams: (array([0, 1]), array([  1, 232]))
unknowns: (array([0, 1]), array([755, 476]))


This code prints the counts of each unique value from the clusters. Remember that 0 means a message was ham and 1 means the message was spam. 

From this output, you can see that 4110 messages fell into the definitely ham group, of which 4071 were actually ham and only 39 were spam. Conversely, of the 233 messages that fell into the definitely spam group, only 1 was actually ham and the rest were spam.

Of course, over 1200 messages fell into the unknown category, so some more advanced analysis would be needed to classify those messages. You might want to look into something like natural language processing to help improve the accuracy of your prediction, and you can use Python and Keras to help out.

### Using the Optimize Module in SciPy
---

When you need to optimize the input parameters for a function, scipy.optimize contains a number of useful methods for optimizing different kinds of functions:

- `minimize_scalar()` and `minimize()` to minimize a function of one variable and many variables, respectively
- `curve_fit()` to fit a function to a set of data
- `root_scalar()` and `root()` to find the zeros of a function of one variable and many variables, respectively
- `linprog()` to minimize a linear objective function with linear inequality and equality constraints


In practice, all of these functions are performing optimization of one sort or another. In this section, you’ll learn about the two minimization functions, minimize_scalar() and minimize().

#### Minimizing a Function With One Variable
---

A mathematical function that accepts one number and results in one output is called a scalar function. It’s usually contrasted with multivariate functions that accept multiple numbers and also result in multiple numbers of output. You’ll see an example of optimizing multivariate functions in the next section.

For this section, your scalar function will be a quartic polynomial, and your objective is to find the minimum value of the function. 

The function is `y = 3x⁴ - 2x + 1.` The function is plotted in the image below for a range of x from 0 to 1:

minimum value of this function at approximately x = 0.55. You can use `minimize_scalar()` to determine the exact x and y coordinates of the minimum. First, `import minimize_scalar()` from `scipy.optimize.` Then, you need to define the objective function to be minimized:

In [31]:
from scipy.optimize import minimize_scalar

def objective_function(x):
    return 3 * x ** 4 - 2 * x + 1

`objective_function()` takes the input x and applies the necessary mathematical operations to it, then returns the result. In the function definition, you can use any mathematical functions you want. The only limit is that the function must return a single number at the end.

Next, use `minimize_scalar()` to find the minimum value of this function. minimize_scalar() has only one required input, which is the name of the objective function definition:

In [33]:
res = minimize_scalar(objective_function)
res

     fun: 0.17451818777634331
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959

The output of `minimize_scalar() is an instance of OptimizeResult.` This class collects together many of the relevant details from the optimizer’s run, including whether or not the optimization was successful and, if successful, what the final result was. 

If the optimization was successful, then fun is the value of the objective function at the optimal value x. You can see from the output that, as expected, `the optimal value for this function was near x = 0.55.`

`functions that have several minima.` In these cases, minimize_scalar() is not guaranteed to find the global minimum of the function. However, `minimize_scalar() has a method keyword argument that you can specify to control the solver that’s used for the optimization.` The SciPy library has three built-in methods for scalar minimization:

- `brent` is an implementation of Brent’s algorithm. This method is the default.
- `golden` is an implementation of the golden-section search. The documentation notes that Brent’s method is usually better.
- `bounded` is a bounded implementation of Brent’s algorithm. It’s useful to limit the search region when the minimum is in a known range.

`When method is either brent or golden, minimize_scalar() takes another argument called bracket. This is a sequence of two or three elements that provide an initial guess for the bounds of the region with the minimum. However, these solvers do not guarantee that the minimum found will be within this range.`

`when method is bounded, minimize_scalar() takes another argument called bounds. This is a sequence of two elements that strictly bound the search region for the minimum. `

Try out the bounded method with the function `y = x⁴ - x².` This function is plotted in the figure below:

In [34]:
# Using the previous example code, you can redefine objective_function() like so:
def objective_function(x):
    return x ** 4 - x ** 2

In [35]:
# Lets try the default method:
res = minimize_scalar(objective_function)
res

     fun: -0.24999999999999994
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 15
     nit: 11
 success: True
       x: 0.7071067853059209

You can see that the optimization was successful. `It found the optimum near x = 0.707` and `y = -1/4.` If you solved for the minimum of the equation analytically, then you’d find the minimum at x = 1/√2, which is extremely close to the answer found by the minimization function. However, `what if you wanted to find the symmetric minimum` at `x = -1/√2`? `You can return the same result by providing the bracket argument to the brent method:`

In [36]:
res = minimize_scalar(objective_function, bracket=(-1, 0))
res

     fun: -0.24999999999999997
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.48e-08 )'
    nfev: 17
     nit: 13
 success: True
       x: 0.7071067809244586

In this code, you add method and bounds as arguments to minimize_scalar(), and you set bounds to be from -1 to 0. The output of this method is as follows:

In [37]:
# Or we can use the golden method:
res = minimize_scalar(objective_function, method="golden")
res

     fun: -0.25
 message: '\nOptimization terminated successfully;\nThe returned value satisfies the termination criteria\n(using xtol = 1.4901161193847656e-08 )'
    nfev: 44
     nit: 39
 success: True
       x: 0.707106784237369

#### Minimizing a Function With Many Variables
---

scipy.optimize also includes the more general minimize(). This function can handle multivariate inputs and outputs and has more complicated optimization algorithms to be able to handle this. In addition, minimize() can handle constraints on the solution to your problem. You can specify three types of constraints:

- `LinearConstraint:` The solution is constrained by taking the inner product of the solution x values with a user-input array and comparing the result to a lower and upper bound.
- `NonlinearConstraint:` The solution is constrained by applying a user-supplied function to the solution x values and comparing the return value with a lower and upper bound.
- `Bounds:` The solution x values are constrained to lie between a lower and upper bound.


Let’s try a demonstration on how to use `minimize().`

 Imagine you’re a stockbroker who’s interested in maximizing the total income from the sale of a fixed number of your stocks. You have identified a particular set of buyers, and for each buyer, you know the price they’ll pay and how much cash they have on hand.

You can phrase this problem as a constrained optimization problem. The objective function is that you want to maximize your income. However, minimize() finds the minimum value of a function, so you’ll need to multiply your objective function by -1 to find the x-values that produce the largest negative number.

There is one constraint on the problem, which is that the sum of the total shares purchased by the buyers does not exceed the number of shares you have on hand. There are also bounds on each of the solution variables because each buyer has an upper bound of cash available, and a lower bound of zero. Negative solution x-values mean that you’d be paying the buyers!

Try out the code below to solve this problem. First, import the modules you need and then set variables to determine the number of buyers in the market and the number of shares you want to sell:

In [38]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

n_buyers = 10
n_shares = 15

Next, create arrays to store the price that each buyer pays, the maximum amount they can afford to spend, and the maximum number of shares each buyer can afford, given the first two arrays. For this example, you can use random number generation in np.random to generate the arrays:

In [39]:
np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)

In this code, you set the seed for NumPy’s random number generators. This function makes sure that each time you run this code, you’ll get the same set of random numbers. It’s here to make sure that your output is the same as the tutorial for comparison.

In line 7, you generate the array of prices the buyers will pay. np.random.random() creates an array of random numbers on the half-open interval [0, 1). The number of elements in the array is determined by the value of the argument, which in this case is the number of buyers.

In line 8, you generate an array of integers on the half-open interval from [1, 4), again with the size of the number of buyers. This array represents the total cash each buyer has available.

In [40]:
# compute the maximum number of shares each buyer can purchase:
n_shares_per_buyer = money_available / prices
print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981]
[1 1 1 3 1 3 3 2 1 1]
[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


The first row is the array of prices, which are floating-point numbers between 0 and 1. This row is followed by the maximum cash available in integers from 1 to 4. Finally, you see the number of shares each buyer can purchase.

Now, you need to create the constraints and bounds for the solver. The constraint is that the sum of the total purchased shares can’t exceed the total number of shares available. This is a constraint rather than a bound because it involves more than one of the solution variables.

To represent this mathematically, you could say that `x[0] + x[1] + ... + x[n] = n_shares,` where `n is the total number of buyers.` More succinctly, you could take the dot or inner product of a vector of ones with the solution values, and constrain that to be equal to n_shares. Remember that LinearConstraint takes the dot product of the input array with the solution values and compares it to the lower and upper bound. You can use this to set up the constraint on n_shares:

In [41]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)

In this code, you use a comprehension to generate a list of tuples for each buyer. The last step before you run the optimization is to define the objective function. Recall that you’re trying to maximize your income. Equivalently, you want to make the negative of your income as large a negative number as possible.

`The income that you generate from each sale is the price that the buyer pays multiplied by the number of shares they’re buying.` Mathematically, you could write this as p`rices[0]*x[0] + prices[1]*x[1] + ... + prices[n]*x[n],` where `n is again the total number of buyers.`

In [44]:
def objective_function(x, prices):
    return -x.dot(prices)

In this code, you define objective_function() to take two arguments. Then you take the dot product of x with prices and return the negative of that value. Remember that you have to return the negative because you’re trying to make that number as small as possible, or as close to negative infinity as possible. Finally, you can call minimize():

In [46]:
bounds = [(0, n) for n in n_shares_per_buyer]

In [47]:
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers),
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
)

In this code, res is an instance of OptimizeResult, just like with minimize_scalar(). As you’ll see, there are many of the same fields, even though the problem is quite different. In the call to minimize(), you pass five arguments:

objective_function: The first positional argument must be the function that you’re optimizing.

x0: The next argument is an initial guess for the values of the solution. In this case, you’re just providing a random array of values between 0 and 10, with the length of n_buyers. For some algorithms or some problems, choosing an appropriate initial guess may be important. However, for this example, it doesn’t seem too important.

args: The next argument is a tuple of other arguments that are necessary to be passed into the objective function. minimize() will always pass the current value of the solution x into the objective function, so this argument serves as a place to collect any other input necessary. In this example, you need to pass prices to objective_function(), so that goes here.

constraints: The next argument is a sequence of constraints on the problem. You’re passing the constraint you generated earlier on the number of available shares.

bounds: The last argument is the sequence of bounds on the solution variables that you generated earlier.

Once the solver runs, you should inspect res by printing it:



In [48]:
res

     fun: -8.783020157087698
     jac: array([-0.77132058, -0.02075195, -0.63364816, -0.74880385, -0.4985069 ,
       -0.22479677, -0.1980629 , -0.76053071, -0.16911089, -0.08833981])
 message: 'Optimization terminated successfully'
    nfev: 165
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([1.29647768, 0.        , 1.57816269, 4.00638948, 2.00598984,
       3.48323773, 0.        , 2.62974258, 0.        , 0.        ])

In this output, you can see message and status indicating the final state of the optimization. For this optimizer, a status of 0 means the optimization terminated successfully, which you can also see in the message. Since the optimization was successful, fun shows the value of the objective function at the optimized solution values. You’ll make an income of $8.78 from this sale.

You can see the values of x that optimize the function in res.x. In this case, the result is that you should sell about 1.3 shares to the first buyer, zero to the second buyer, 1.6 to the third buyer, 4.0 to the fourth, and so on.

You should also check and make sure that the constraints and bounds that you set are satisfied. You can do this with the following code:

In [49]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)

The total number of shares is: 14.999999999999993
Leftover money for each buyer: [0.         1.         0.         0.         0.         2.21697984
 3.         0.         1.         1.        ]
